# 3.1 Sockets

A **socket** in Python refers to the endpoint of a communication channel most generally. That is, if we have a connection between two devices, we have a socket associated with each device. We can have many different types of sockets depending on the channel of communication, e.g internet or bluetooth socket.  

## TCP Internet Socket

In [2]:
import socket

# Creating socket object for a TCP internet connection,
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

_socket.AF_INET_ specifies that we have an **internet connection** while _socket.SOCK_STREAM_ tells the socket to use TCP as our protocol.